# Tải thư viện và load model

In [1]:
!pip install --upgrade pip
!pip install diffusers==0.30.0 safetensors sentencepiece transformers accelerate datasets bitsandbytes deepspeed gradio==3.48.0 pandas invisible-watermark
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!pip install --upgrade diffusers transformers accelerate
!pip install --upgrade peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 113.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 82.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 20.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 103.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━

In [2]:
!pip install pytorch-fid --quiet
!pip install openai-clip --quiet
!pip install hpsv2 --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm
import os
import glob
import clip

2025-11-08 16:39:57.919175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762619998.123398      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762619998.185986      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
base_path = "runwayml/stable-diffusion-v1-5"
unet_path = "cosmicman/CosmicMan-SD"

# Load model.
unet = UNet2DConditionModel.from_pretrained(unet_path, torch_dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(base_path, unet=unet, torch_dtype=torch.float16, variant="fp16", safety_checker=None).to("cuda")
pipe.scheduler = EulerDiscreteScheduler.from_pretrained(base_path, subfolder="scheduler", torch_dtype=torch.float16)

config.json: 0.00B [00:00, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 2000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

...

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# FID

In [12]:
REAL_DIR_FULLBODY = "/kaggle/input/cosmicman-images/Cosmicman_full_body/real_images"
REAL_DIR_FACE = "/kaggle/input/mm-celeba-hq-dataset/Cosmicman_face/real_images"
GEN_DIR_COSMICMAN_FULLBODY = "/kaggle/input/cosmicman-images/Cosmicman_full_body/generated_images"
GEN_DIR_SD_FULLBODY = "/kaggle/input/cosmicman-images/SD_full_body/generated_images_SD"
GEN_DIR_COSMICMAN_FACE = "/kaggle/input/mm-celeba-hq-dataset/Cosmicman_face/generated_images"
GEN_DIR_SD_FACE = "/kaggle/input/mm-celeba-hq-dataset/SD_face/generated_images"
PROMPTS_DIR_FULLBODY = "/kaggle/input/cosmicman-images/prompts/prompts"
PROMPTS_DIR_FACE = "/kaggle/input/mm-celeba-hq-dataset/prompts_face/prompts"

## SD

In [13]:
!python -m pytorch_fid {REAL_DIR_FULLBODY} {GEN_DIR_SD_FULLBODY} --device cuda

Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|███████████████████████████████████████| 91.2M/91.2M [00:00<00:00, 199MB/s]
100%|███████████████████████████████████████████| 20/20 [00:33<00:00,  1.66s/it]
FID:  149.62692278648677


In [14]:
!python -m pytorch_fid {REAL_DIR_FACE} {GEN_DIR_SD_FACE} --device cuda

100%|███████████████████████████████████████████| 10/10 [00:15<00:00,  1.59s/it]
FID:  138.41387115510275


## Cosmicman

In [15]:
!python -m pytorch_fid {REAL_DIR_FULLBODY} {GEN_DIR_COSMICMAN_FULLBODY} --device cuda

100%|███████████████████████████████████████████| 20/20 [00:31<00:00,  1.60s/it]
FID:  138.21604126432362


In [16]:
!python -m pytorch_fid {REAL_DIR_FACE} {GEN_DIR_COSMICMAN_FACE} --device cuda

100%|███████████████████████████████████████████| 10/10 [00:18<00:00,  1.80s/it]
FID:  120.14339250740295


# CLIPScore

## Trích folder prompts

In [7]:
# START_INDEX = 0
# END_INDEX = 500

# # Đường dẫn dataset
# IMAGE_DIR = "/kaggle/input/d/kashyapkvh/mm-celeba-hq-dataset/MM-CELEBA-HQ/images"
# TEXT_DIR = "/kaggle/input/d/kashyapkvh/mm-celeba-hq-dataset/MM-CELEBA-HQ/text/celeba-caption"

In [8]:
# from transformers import CLIPTokenizer
# PROMPT_DIR = "./prompts"
# os.makedirs(PROMPT_DIR, exist_ok=True)

# print("Đang tải tokenizer...")
# try:
#     tokenizer = CLIPTokenizer.from_pretrained(base_path, subfolder="tokenizer")
#     print("Tải tokenizer thành công.")
# except Exception as e:
#     print(f"Lỗi khi tải tokenizer: {e}")
#     print("Vui lòng đảm bảo bạn có kết nối mạng nếu tokenizer chưa được cache.")
#     # Dừng cell nếu không tải được tokenizer
#     raise e

Đang tải tokenizer...
Tải tokenizer thành công.


In [9]:
# print("Đang quét đường dẫn ảnh...")
# all_image_paths = sorted(glob.glob(os.path.join(IMAGE_DIR, "*.jpg")))
# image_paths = all_image_paths[START_INDEX:END_INDEX]

# # --- 4. Vòng lặp trích xuất và lưu prompt ---
# print(f"Bắt đầu trích xuất {len(image_paths)} prompt...")
# for i, image_path in enumerate(tqdm(image_paths), start=START_INDEX):
#     # Tên file prompt output
#     prompt_save_filename = f"{i:05d}.txt"
#     prompt_save_path = os.path.join(PROMPT_DIR, prompt_save_filename)
    
#     try:
#         # --- A. Tìm file text tương ứng ---
#         image_filename = os.path.basename(image_path)
#         text_filename = os.path.splitext(image_filename)[0] + ".txt"
#         text_path = os.path.join(TEXT_DIR, text_filename)

#         # --- B. Đọc tất cả captions ---
#         with open(text_path, 'r') as f:
#             captions = [line.strip() for line in f.readlines() if line.strip()]

#         # --- C. Lọc ra caption tốt nhất ---
#         valid_captions = []
#         for cap in captions:
#             # Tokenize để kiểm tra độ dài
#             inputs = tokenizer(cap, return_tensors="pt", padding=False, truncation=False)
#             token_count = inputs.input_ids.shape[1]
            
#             # Giới hạn của CLIP là 77 tokens
#             if token_count <= 77:
#                 valid_captions.append(cap)
        
#         if not valid_captions:
#             raise Exception(f"Không tìm thấy caption nào <= 77 tokens.")

#         # Chọn prompt dài nhất (đầy đủ ý nhất)
#         prompt = max(valid_captions, key=len)

#         # --- D. Lưu prompt đã chọn ---
#         with open(prompt_save_path, 'w') as f:
#             f.write(prompt)

#     except Exception as e:
#         print(f"Lỗi ở ảnh {i} (path: {image_path}): {e}. Bỏ qua file này.")

# print(f"Hoàn tất việc trích xuất và lưu {len(image_paths)} prompt vào thư mục {PROMPT_DIR}!")

Đang quét đường dẫn ảnh...
Bắt đầu trích xuất 500 prompt...


100%|██████████| 500/500 [00:04<00:00, 119.62it/s]

Hoàn tất việc trích xuất và lưu 500 prompt vào thư mục ./prompts!


In [10]:
# !zip -r prompts.zip ./prompts

  adding: prompts/ (stored 0%)
  adding: prompts/00089.txt (deflated 22%)
  adding: prompts/00122.txt (deflated 9%)
  adding: prompts/00184.txt (deflated 29%)
  adding: prompts/00171.txt (deflated 9%)
  adding: prompts/00413.txt (deflated 26%)
  adding: prompts/00148.txt (deflated 6%)
  adding: prompts/00220.txt (deflated 29%)
  adding: prompts/00433.txt (deflated 20%)
  adding: prompts/00380.txt (deflated 15%)
  adding: prompts/00104.txt (deflated 23%)
  adding: prompts/00128.txt (deflated 21%)
  adding: prompts/00275.txt (deflated 2%)
  adding: prompts/00151.txt (deflated 27%)
  adding: prompts/00411.txt (deflated 30%)
  adding: prompts/00302.txt (deflated 24%)
  adding: prompts/00188.txt (deflated 27%)
  adding: prompts/00081.txt (deflated 17%)
  adding: prompts/00285.txt (deflated 20%)
  adding: prompts/00183.txt (deflated 20%)
  adding: prompts/00065.txt (deflated 19%)
  adding: prompts/00007.txt (deflated 23%)
  adding: prompts/00483.txt (deflated 29%)
  adding: prompts/00498.txt

In [ ]:
# # --- Các hằng số ---
# NUM_IMAGES = 1000
# DATASET_ID = "Marqo/deepfashion-multimodal"
# SPLIT_NAME = "data"

# KAGGLE_WORKING_DIR = "/kaggle/working/"
# PROMPTS_DIR = os.path.join(KAGGLE_WORKING_DIR, "prompts")

# os.makedirs(PROMPTS_DIR, exist_ok=True)

# print(f"Đang tải {NUM_IMAGES} mẫu từ '{DATASET_ID}' (Yêu cầu Internet)...")
# try:
#     dataset = load_dataset(
#         DATASET_ID,
#         split=f'{SPLIT_NAME}[:{NUM_IMAGES}]'
#     )
    
#     print(f"Đã tải thành công {len(dataset)} mẫu.")
#     print(f"Đang lưu {len(dataset)} prompts vào các file .txt riêng lẻ...")

#     for i, example in enumerate(tqdm(dataset)):
#         # Dùng .get('text') để tránh lỗi nếu key 'text' không tồn tại
#         prompt_text = example.get('text') 

#         # Tên file phải khớp với tên file ảnh 
#         file_name = f"{i:05d}.txt"
#         file_path = os.path.join(PROMPTS_DIR, file_name)

#         # Ghi file
#         with open(file_path, "w", encoding="utf-8") as f:
#             if prompt_text and isinstance(prompt_text, str):
#                 f.write(prompt_text.strip())
#             else:
#                 # Ghi file rỗng nếu prompt là None, rỗng, hoặc không phải string
#                 f.write("")

#     print(f"\nHoàn tất! Đã lưu {len(dataset)} file .txt vào thư mục '{PROMPTS_DIR}'.")
#     print("Bạn có thể thấy các file này trong tab 'Data' -> 'Output' -> 'kaggle/working/prompts_dataset'")

# except Exception as e:
#     print(f"\n--- LỖI KHI TẢI DATASET ---")

In [ ]:
# import shutil
# import os

# # Đường dẫn đến thư mục bạn đã tạo
# PROMPTS_DIR = "/kaggle/working/prompts"

# # Tên file zip bạn muốn tạo, nó sẽ được lưu ở /kaggle/working/
# OUTPUT_ZIP_FILE = "/kaggle/working/prompts" 

# print(f"Đang nén thư mục '{PROMPTS_DIR}'...")

# try:
#     # Lệnh này sẽ tạo file 'prompts_archive.zip'
#     # chứa toàn bộ nội dung của thư mục 'prompts_dataset'
#     shutil.make_archive(
#         base_name=OUTPUT_ZIP_FILE,  # Tên file zip (không cần .zip)
#         format='zip',               # Định dạng nén
#         root_dir="/kaggle/working/",# Thư mục gốc để bắt đầu nén
#         base_dir="prompts"  # Thư mục cụ thể cần nén
#     )
#     print(f"Đã tạo file nén thành công: {OUTPUT_ZIP_FILE}.zip")
# except Exception as e:
#     print(f"Lỗi khi nén file: {e}")

In [17]:
# Tải model CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Đang tải model CLIP (ViT-B/32) lên {device}...")
model, preprocess = clip.load("ViT-B/32", device=device)
print("Tải model CLIP thành công.")

Đang tải model CLIP (ViT-B/32) lên cuda...


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 120MiB/s]


Tải model CLIP thành công.


In [18]:
def calculate_clip_score_programmatically(image_dir, text_dir, batch_size=64):

    # Lấy danh sách file ảnh và text, đảm bảo chúng được sắp xếp
    image_files = sorted(glob.glob(os.path.join(image_dir, "*.png")))
    text_files = sorted(glob.glob(os.path.join(text_dir, "*.txt")))

    total_score = 0.0
    num_files = len(image_files)

    with torch.no_grad():
        for i in tqdm(range(0, num_files, batch_size), desc=f"Tính CLIPScore cho {os.path.basename(image_dir)}"):
            # Lấy các file cho batch hiện tại
            batch_img_paths = image_files[i:i+batch_size]
            batch_txt_paths = text_files[i:i+batch_size]

            # --- Xử lý Batch Ảnh ---
            images_preprocessed = []
            for img_path in batch_img_paths:
                image = Image.open(img_path)
                images_preprocessed.append(preprocess(image))
            image_input = torch.stack(images_preprocessed).to(device)

            # --- Xử lý Batch Text ---
            texts = []
            for txt_path in batch_txt_paths:
                with open(txt_path, 'r', encoding='utf-8') as f:
                    texts.append(f.read().strip())

            # Tự động PADDING và TRUNCATION (cắt bớt nếu > 77)
            text_input = clip.tokenize(texts, truncate=True).to(device)

            # --- Tính toán đặc trưng ---
            image_features = model.encode_image(image_input)
            text_features = model.encode_text(text_input)

            # Chuẩn hóa L2
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

            # Tính điểm
            # (Thư viện clip-score dùng W=2.5, dựa trên logit_scale của CLIP)
            similarity = (image_features * text_features).sum(dim=1)
            batch_scores = similarity * model.logit_scale.exp() * 2.5

            total_score += batch_scores.sum().item()

    # Tính điểm trung bình
    mean_score = total_score / num_files
    return mean_score

## ẢNH GỐC

In [19]:
print("\n--- 1. Bắt đầu tính CLIPScore cho ảnh gốc (fullbody) ---")
score_real = calculate_clip_score_programmatically(REAL_DIR_FULLBODY, PROMPTS_DIR_FULLBODY)

print(f"Ảnh gốc:   {score_real:.4f}")


--- 1. Bắt đầu tính CLIPScore cho ảnh gốc (fullbody) ---


Tính CLIPScore cho real_images: 100%|██████████| 16/16 [00:43<00:00,  2.71s/it]

Ảnh gốc:   70.5860


In [20]:
print("\n--- 1. Bắt đầu tính CLIPScore cho ảnh gốc (face) ---")
score_real = calculate_clip_score_programmatically(REAL_DIR_FACE, PROMPTS_DIR_FACE)

print(f"Ảnh gốc:   {score_real:.4f}")


--- 1. Bắt đầu tính CLIPScore cho ảnh gốc (face) ---


Tính CLIPScore cho real_images: 100%|██████████| 8/8 [00:26<00:00,  3.34s/it]

Ảnh gốc:   65.1720


## SD 

In [21]:
print("\n--- 1. Bắt đầu tính CLIPScore cho model SD (full body) ---")
score_sd = calculate_clip_score_programmatically(GEN_DIR_SD_FULLBODY, PROMPTS_DIR_FULLBODY)

print(f"Model SD:   {score_sd:.4f}")


--- 1. Bắt đầu tính CLIPScore cho model SD (full body) ---


Tính CLIPScore cho generated_images_SD: 100%|██████████| 16/16 [00:49<00:00,  3.12s/it]

Model SD:   71.1840


In [22]:
print("\n--- 1. Bắt đầu tính CLIPScore cho model SD (face) ---")
score_sd = calculate_clip_score_programmatically(GEN_DIR_SD_FACE, PROMPTS_DIR_FACE)

print(f"Model SD:   {score_sd:.4f}")


--- 1. Bắt đầu tính CLIPScore cho model SD (face) ---


Tính CLIPScore cho generated_images: 100%|██████████| 8/8 [00:23<00:00,  2.94s/it]

Model SD:   68.6680


## Cosmicman

In [23]:
print("\n--- 1. Bắt đầu tính CLIPScore cho model Cosmicman (full body) ---")
score_cosmicman = calculate_clip_score_programmatically(GEN_DIR_COSMICMAN_FULLBODY, PROMPTS_DIR_FULLBODY)

print(f"Model Cosmicman:   {score_cosmicman:.4f}")


--- 1. Bắt đầu tính CLIPScore cho model Cosmicman (full body) ---


Tính CLIPScore cho generated_images: 100%|██████████| 16/16 [00:48<00:00,  3.06s/it]

Model Cosmicman:   73.3420


In [24]:
print("\n--- 1. Bắt đầu tính CLIPScore cho model Cosmicman (face) ---")
score_cosmicman = calculate_clip_score_programmatically(GEN_DIR_COSMICMAN_FACE, PROMPTS_DIR_FACE)

print(f"Model Cosmicman:   {score_cosmicman:.4f}")


--- 1. Bắt đầu tính CLIPScore cho model Cosmicman (face) ---


Tính CLIPScore cho generated_images: 100%|██████████| 8/8 [00:23<00:00,  2.94s/it]

Model Cosmicman:   70.6800
